In [1]:
import torch 
import pandas as pd
import torch.nn as nn
import time
import io
import joblib
import argparse

from torchsummary import summary
from tqdm import tqdm
from tabulate import tabulate
from sklearn.preprocessing import *

from AutoCleanse.utils import *
from AutoCleanse.dataloader import PlainDataset, DataLoader
from AutoCleanse.autoencoder import *
from AutoCleanse.loss_model import loss_CEMSE
from AutoCleanse.preprocessor import Preprocessor
from AutoCleanse.anonymize import anonymize
from AutoCleanse.bucketfs_client import *

Device configuration

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

## Setup directory path

In [3]:
import os
PROJECT_DIR = os.getcwd()
os.chdir(PROJECT_DIR)
DATASET_DIR = os.path.join(PROJECT_DIR,'dataset')
EVAL_DIR = os.path.join(PROJECT_DIR,'evaluate')

## Preparing data

Load dataframe and group features by their type

In [4]:
df = pd.read_csv(os.path.join(DATASET_DIR,'adult.csv')).drop(columns=['fnlwgt','income'])
continous_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns.tolist()
og_columns = df.columns.to_list()
df = df[continous_columns+categorical_columns]

Data preprocessing

In [5]:
scaler = MinMaxScaler()
onehotencoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
preprocessor = Preprocessor(scaler,onehotencoder)

X_train,X_val,X_test = preprocessor.split(df=df,
                                        train_ratio=0.7,
                                        val_ratio=0.15,
                                        test_ratio=0.15,
                                        random_seed=42)
X_dirty = replace_with_nan(X_test,0.2,42)

X_train = preprocessor.fit_transform(input_df=X_train,
                                    continous_columns=continous_columns,
                                    categorical_columns=categorical_columns)

X_val = preprocessor.transform(input_df=X_val,    
                               continous_columns=continous_columns,
                               categorical_columns=categorical_columns)                          

X_test = preprocessor.transform(input_df=X_test,   
                                continous_columns=continous_columns,
                                categorical_columns=categorical_columns)  

X_dirty = preprocessor.transform(input_df=X_dirty,   
                                continous_columns=continous_columns,
                                categorical_columns=categorical_columns)
categories = preprocessor.encoder.categories_

/home/tung/development/AutoCleanse/AutoCleanse/utils.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[48, 28, 38, nan, nan, 27, 25, 20, 39, 52, 40, 35, 55, 62, nan, 17, 22, nan, 17, 51, 20, nan, 44, 19, 21, 77, 46, nan, 21, 48, 39, 34, nan, nan, 32, 52, 30, 20, 46, 30, 20, 41, 39, 21, 47, nan, 28, nan, nan, 21, 23, 24, 35, nan, nan, 49, 65, 42, nan, 21, nan, 23, 46, 22, 44, 30, 29, 56, nan, 40, 32, 25, nan, nan, nan, 42, 25, 39, 28, 39, 31, nan, 59, 28, 48, 38, nan, 20, 46, 38, 64, nan, 44, 29, 48, 47, 46, 26, nan, 47, 18, 50, nan, 28, 23, 33, 18, 39, 30, 18, 36, 37, 23, 36, 19, 34, nan, 42, nan, 48, 33, 49, 24, nan, 27, 53, nan, 43, 42, 66, 32, nan, 36, 27, 37, 36, 29, 26, 23, 37, 74, 42, 29, nan, 27, 21, 41, 60, 38, 62, 37, 55, 67, 48, 53, 23, 59, 37, 34, 37, 48, 25, 47, 49, nan, 30, 28, 44, 22, 30, nan, 18, 51, 30, 28, 32, 51, 33, 39, 68, 24, 52, 59, 33, 55, 27, nan, 31, 23, 23, 60, 51, 18, 42, nan, 22, 52,

In [6]:
X_dirty

,age,education.num,capital.gain,capital.loss,hours.per.week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
28296,0.424658,0.266667,0.000000,0.000000,9208.575629,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28217,0.150685,0.533333,0.000000,0.000000,0.397959,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8054,0.287671,0.266667,0.000000,0.000000,0.397959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4223,-8742.361300,8561.805903,-5708.054784,0.000000,0.193878,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
22723,8448.118926,0.666667,7859.623854,0.000000,0.846939,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21656,0.109589,6693.105868,0.000000,7976.678645,0.346939,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28318,0.027397,0.400000,0.000000,2640.493095,0.397959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32547,0.301370,0.733333,-8312.431233,-4863.080020,-7774.083226,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9375,0.342466,-7669.257613,0.000000,0.000000,0.397959,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


(Optional) Check save/load function of preprocessor

Both functiona take in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **preprocessor_main.pkl**
    - Save/load location: can either be "local" to save/load in the home folder or "bucketfs" to save/load to/from Exasol BucketFS

In [ ]:
preprocessor.save("main","local")
preprocessor.save("main","bucketfs")
preprocessor = Preprocessor(scaler=MinMaxScaler(),encoder=OneHotEncoder(sparse=False))
preprocessor2.load("main","local")
preprocessor2.load("main","bucketfs")

Convert dataframes into datasets, and create dataloaders

In [7]:
batch_size = 64

In [8]:
train_dataset = PlainDataset(X_train)
val_dataset = PlainDataset(X_val)
test_dataset = PlainDataset(X_test)
dirty_dataset = PlainDataset(X_dirty)

def custom_collate_fn(batch):
    tensor_data = torch.stack([item[0] for item in batch])
    indices = [item[1] for item in batch]
    return tensor_data, indices

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
dirty_loader = DataLoader(dirty_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

## Instantiate model

In [9]:
layers = [X_train.shape[1],1024,128]   
wlc = (1,1) 

autoencoder = Autoencoder(layers=layers,dropout_enc=[(0,0.0)],dropout_dec=[(0,0.1)], batch_norm=True, \
                          learning_rate=1e-4,weight_decay=1e-5,l1_strength=1e-5,l2_strength=1e-5)

In [ ]:
summary(autoencoder.to(device),torch.tensor(X_train.values).float().to(device).shape[1:])

(Optional) Model can be loaded from checkpoint after instantiation

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [ ]:
autoencoder.load("local","main")

### Train the model

In [10]:
autoencoder.train_model(
      patience=10,
      num_epochs=100,
      batch_size=batch_size,
      train_loader=train_loader,
      val_loader=val_loader,
      continous_columns=continous_columns, 
      categorical_columns=categorical_columns, 
      categories=categories,
      device=device,
      wlc=wlc)

Epoch [1/100], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 34.59it/s]


Epoch [1/100], Training Loss: 6.51186028
Epoch [1/100], Validation Loss: 2.39420251
Epoch [1/100], Training CE Loss: 6.45458449
Epoch [1/100], Validation CE Loss: 2.35875442
Epoch [1/100], Training MSE Loss: 0.05727580
Epoch [1/100], Validation MSE Loss: 0.03544809
Epoch [1/100], Training Loss Comp: 6.51186028
Epoch [1/100], Validation Loss Comp: 2.39420251
Epoch [1/100]: Learning Rate = [0.0001]



Epoch [2/100], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 34.43it/s]


Epoch [2/100], Training Loss: 1.45376648
Epoch [2/100], Validation Loss: 0.82513203
Epoch [2/100], Training CE Loss: 1.42414442
Epoch [2/100], Validation CE Loss: 0.79886463
Epoch [2/100], Training MSE Loss: 0.02962206
Epoch [2/100], Validation MSE Loss: 0.02626741
Epoch [2/100], Training Loss Comp: 1.45376648
Epoch [2/100], Validation Loss Comp: 0.82513203
Epoch [2/100]: Learning Rate = [0.0001]



Epoch [3/100], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 34.51it/s]


...

Epoch [50/100], Training Loss: 0.00461912
Epoch [50/100], Validation Loss: 0.01870091
Epoch [50/100], Training CE Loss: 0.00186711
Epoch [50/100], Validation CE Loss: 0.01601502
Epoch [50/100], Training MSE Loss: 0.00275201
Epoch [50/100], Validation MSE Loss: 0.00268588
Epoch [50/100], Training Loss Comp: 0.00461912
Epoch [50/100], Validation Loss Comp: 0.01870091
Epoch [50/100]: Learning Rate = [1.0000000000000002e-06]



Epoch [51/100], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 34.52it/s]


Epoch [51/100], Training Loss: 0.00432375
Epoch [51/100], Validation Loss: 0.01843652
Epoch [51/100], Training CE Loss: 0.00187900
Epoch [51/100], Validation CE Loss: 0.01588091
Epoch [51/100], Training MSE Loss: 0.00244475
Epoch [51/100], Validation MSE Loss: 0.00255561
Epoch [51/100], Training Loss Comp: 0.00432375
Epoch [51/100], Validation Loss Comp: 0.01843652
Epoch [51/100]: Learning Rate = [1.0000000000000002e-06]



Epoch [52/100], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 34.75it/s]

Epoch [52/100], Training Loss: 0.00423427
Epoch [52/100], Validation Loss: 0.01956219
Epoch [52/100], Training CE Loss: 0.00178316
Epoch [52/100], Validation CE Loss: 0.01700943
Epoch [52/100], Training MSE Loss: 0.00245111
Epoch [52/100], Validation MSE Loss: 0.00255276
Epoch [52/100], Training Loss Comp: 0.00423427
Epoch [52/100], Validation Loss Comp: 0.01956219
Epoch [52/100]: Learning Rate = [1.0000000000000002e-06]

Early stopping triggered. Stopping training.


(Optional) Model can be saved after training

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [11]:
autoencoder.save("local","main")

Saved weight to autoencoder_main.pth


### Use trained model to clean data

In [16]:
cleaned_data = autoencoder.clean(dirty_loader=dirty_loader,
                                test_loader=test_loader,
                                df=X_dirty,
                                batch_size=batch_size,
                                continous_columns=continous_columns, 
                                categorical_columns=categorical_columns, 
                                og_columns=og_columns,
                                onehotencoder=preprocessor.encoder, 
                                scaler=preprocessor.scaler,
                                device=device) 

Clean progress: 100%|██████████| 76/76 [00:04<00:00, 18.33it/s]


MAE: 0.00116814

MSE: 0.00012697


In [13]:
# original data
print(tabulate(df.loc[[28296,28217,8054,4223,22723],og_columns],headers=og_columns,tablefmt="simple",maxcolwidths=[None, 4]))

         age  workclass         education      education.num  marital.status      occupation         relationship    race                sex       capital.gain    capital.loss    hours.per.week  native.country
-----  -----  ----------------  -----------  ---------------  ------------------  -----------------  --------------  ------------------  ------  --------------  --------------  ----------------  ----------------
28296     48  ?                 9th                        5  Separated           ?                  Not-in-family   Amer-Indian-Eskimo  Female               0               0                20  United-States
28217     28  ?                 HS-grad                    9  Separated           ?                  Unmarried       White               Female               0               0                40  United-States
 8054     38  Self-emp-not-inc  9th                        5  Divorced            Craft-repair       Not-in-family   White               Male                 0 

In [17]:
# cleaned data
print(tabulate(cleaned_data.loc[[28296,28217,8054,4223,22723]],headers=cleaned_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 4]))

         age  workclass         education      education.num  marital.status      occupation         relationship    race                sex       capital.gain    capital.loss    hours.per.week  native.country
-----  -----  ----------------  -----------  ---------------  ------------------  -----------------  --------------  ------------------  ------  --------------  --------------  ----------------  ----------------
28296     48  ?                 9th                        6  Separated           ?                  Not-in-family   Amer-Indian-Eskimo  Female           -4947             230                30  United-States
28217     31  ?                 HS-grad                    9  Separated           ?                  Unmarried       White               Female           -2421              -7                37  United-States
 8054     36  Self-emp-not-inc  9th                        5  Divorced            Craft-repair       Not-in-family   White               Male              2361 

### Use trained model to anonymize data

In [ ]:
anonymized_data = autoencoder.anonymize(df=X_test,
                                        data_loader=test_loader,
                                        batch_size=batch_size,
                                        device=device)

In [ ]:
# anonymized data
print(tabulate(anonymized_data.round(decimals=4).iloc[:5,:32],headers=anonymized_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 6]))